In [1]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade gensim

  Using cached https://files.pythonhosted.org/packages/e2/a6/ec7a2b8bb0a0cd864e437e1984398893f959592dc0d702ffffb1683fe7e3/gensim-3.8.1-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/fc/3c/73b86551889c173627f2b5aafa5df89d19c02056c9a676d1490a01d09c70/scipy-1.3.3-cp36-cp36m-win_amd64.whl
Processing c:\users\jungs\appdata\local\pip\cache\wheels\ab\10\93\5cff86f5b721d77edaecc29959b1c60d894be1f66d91407d28\smart_open-1.9.0-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/23/10/c0b78c27298029e4454a472a1919bde20cb182dab1662cec7f2ca1dcc523/boto-2.49.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/67/17/567f679dac6ec93611ce05637094e9736afa40e41c9ed0aaefd90de6f7e8/boto3-1.10.28-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/16/8a/1fc3dba0c4923c2a76e1ff0d52b305c44606da63f718d14d3231e21c51b0/s3transfer-0.2.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8e

In [3]:
pip install utils

  Using cached https://files.pythonhosted.org/packages/9b/de/9ffaf89be661b32d1e0cff05e1af5e4fc2d608c47498975e94aca219aed4/utils-0.9.0-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install nltk

  Using cached https://files.pythonhosted.org/packages/f6/1d/d925cfb4f324ede997f6d47bea4d9babba51b49e87a767c170b77005889d/nltk-3.4.5.zip
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449910 sha256=75e9c4368592e9abf9e8de7ec97dec607e7e90b31399e7073ad814f468660d09
  Stored in directory: C:\Users\jungs\AppData\Local\pip\Cache\wheels\96\86\f6\68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
Note: you may need to restart the kernel to use updated packages.


In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jungs\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from matplotlib import font_manager, rc
font_fname = 'c:/windows/fonts/gulim.ttc'     # A font of your choice
font_name = font_manager.FontProperties(fname=font_fname).get_name()
rc('font', family=font_name)

In [9]:
# import torch
from time import time
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

In [ ]:
def read_data(filename):
    with open(filename, 'r', encoding='UTF8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data
train_data = read_data('E:\\nlp\\data\\ho_train.txt')
test_data = read_data('E:\\nlp\\data\\ho_test.txt')

In [ ]:
print(len(train_data))
print(len(train_data[0]))
print(len(test_data))
print(len(test_data[0]))

In [ ]:
start = time()
import konlpy
from konlpy.tag import Okt
from konlpy.utils import pprint
pos_tagger = Okt()
def tokenize(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]
train_docs = [(tokenize(row[0]), row[1]) for row in train_data]
test_docs = [(tokenize(row[0]), row[1]) for row in test_data]

end = time()
print('Time: {:f}s'.format(end-start))
tokens = [t for d in train_docs for t in d[0]]
print(len(tokens))

In [ ]:
import nltk
text = nltk.Text(tokens, name='NMSC')

# 전체 토큰의 개수
print(len(text.tokens))

# 중복을 제외한 토큰의 개수
print(len(set(text.tokens)))            

# 출현 빈도가 높은 상위 토큰 10개
pprint(text.vocab().most_common(10))

In [ ]:
selected_words = [f[0] for f in text.vocab().most_common(1000)]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

train_x = [term_frequency(d) for d, _ in train_docs]
test_x = [term_frequency(d) for d, _ in test_docs]
train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

In [ ]:
import numpy as np

x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')

In [ ]:
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras import optimizers
from tensorflow.python.keras import losses
from tensorflow.python.keras import metrics

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(1000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

model.fit(x_train, y_train, epochs=10, batch_size=512)
results = model.evaluate(x_test, y_test)

In [ ]:
results

In [ ]:
def predict_pos_neg(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(model.predict(data))
    if(score > 0.5):
        print("[{}]는 {:.2f}% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^\n".format(review, score * 100))
    else:
        print("[{}]는 {:.2f}% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;\n".format(review, (1 - score) * 100))

In [ ]:
predict_pos_neg("올해 최고의 영화! 세 번 넘게 봐도 질리지가 않네요.")
predict_pos_neg("배경 음악이 영화의 분위기랑 너무 안 맞았습니다. 몰입에 방해가 됩니다.")
predict_pos_neg("주연 배우가 신인인데 연기를 진짜 잘 하네요. 몰입감 ㅎㄷㄷ")
predict_pos_neg("믿고 보는 감독이지만 이번에는 아니네요")
predict_pos_neg("주연배우 때문에 봤어요")
